# Torchvision CNN's for Smoke Classification
In this notebook, torchvision models are adopted for smoke classification using the SmokeNet Dataset. Initial research brought VGG16, inception and resnet to our attention, so these models were focused on to begin with.

The primary goal of this effort is to develop working knowledge with the Pytorch framework for machine learning and AI. 

In [8]:
#################### Imports #########################
from __future__ import print_function, division
import warnings
import pandas as pd
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.models as models
import torch.nn as nn
from torch.optim import lr_scheduler
import time
import numpy as np
from torch.autograd import Variable
import os
import torch
# Ignore warnings
warnings.filterwarnings("ignore")
# interactive mode
plt.ion()   
multiGPU = False

In [9]:
################ Custom Data Class #####################
""" 
The data set is composed of 1164 cloud images (class 1), 1009 dust (class 2), 1002 haze (class 3), 1027 land (class 4), 
1007 seaside (class 5), and 1016 smoke (class 0, the target.)

The RGB images total to 6225 and are 256x256 with a resolution of 1km.

The dataclass inherits from Pytroches Dataset class in order to form the necassary strucutre for using it with a
dataloader.
"""

class SmokeNetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations) # should be 6225 (ie. no. images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):                                                 
            idx = idx.tolist()

        image_path = os.path.join(self.root_dir,
                                self.annotations.iloc[idx, 0])   # get name of the image at idx
        image = io.imread(image_path)
        y_label = torch.tensor(int(self.annotations.iloc[idx, 1]))

        if self.transform:
          image = self.transform(image)
        
        return (image, y_label)
        

In [22]:
# go to the data in google drive
%cd '/content/drive/My Drive/SmokeNet'

/content/drive/My Drive/SmokeNet


In [23]:
################ Initialisation inlc. data processing #####################

# hyperperameters
num_classes = 6
learning_rate = 1e-3
batch_size = 32 
num_epochs = 20

# pack intended data augmentation transforms into a single object
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5]) # SmokeNet used this
    ])

# load data from SmokeNet directory 
dataset = SmokeNetDataset(csv_file = 'SmokeNetData.csv', root_dir = 'SmokePics',
                          transform = data_transform)

# split the data into a  70/30 train/test split
train_set, test_set = torch.utils.data.random_split(dataset, [4357, 1868])    

# create the data loaders for both datasets
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# create dictionaries to contain both dataloaders 
dataloaders = {"train" : train_loader, "val" : test_loader}
datasets = {"train": train_set, "val": test_set}


In [24]:
# define variables if GPU is to be used
if torch.cuda.is_available():
    use_gpu = True
    print("Using GPU")
else:
    use_gpu = False
FloatTensor = torch.cuda.FloatTensor if use_gpu else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_gpu else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_gpu else torch.ByteTensor
Tensor = FloatTensor

Using GPU


In [25]:
################ Function for Training the Model #####################

def train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs):
    since = time.time()

    # initialise best weights as the random untrained ones
    best_model_wts = model.state_dict()
    # initialise best accuracy for comparing epochs and saving best weights
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:     
            since_epoch = time.time()
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
    
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                inputs = Variable(inputs.type(Tensor))
                labels = Variable(labels.type(LongTensor))

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                # statistics
                running_loss += loss.data # changed from loss.data[0] because an error saying invalid index of 0-dim tensor
                running_corrects += float(torch.sum(preds == labels.data))

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects / len(datasets[phase])

            time_elapsed_epoch = time.time() - since_epoch
            print('{} Loss: {:.4f} Acc: {:.4f} in {:.0f}m {:.0f}s'.format(
                phase, epoch_loss, epoch_acc, time_elapsed_epoch // 60, time_elapsed_epoch % 60))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # keep the best model weights
    model.load_state_dict(best_model_wts)
    
    return model

In [8]:
################# Initialse the models ########################

"""
# import the model from torchvision
model = torchvision.models.googlenet(pretrainted=False)
"""

criterion = nn.CrossEntropyLoss()

################ Large Resnet
resnet152 = models.resnet152(pretrained=False)
num_ftrs = resnet152.fc.in_features
resnet152.fc = nn.Linear(num_ftrs, 120)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(resnet152.parameters(), lr=learning_rate)                  

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


################### smaller resnet
resnet18 = models.resnet18(pretrained=False)
optimizer_ft_18 = optim.Adam(resnet18.parameters(), lr=learning_rate)                  
exp_lr_scheduler_18 = lr_scheduler.StepLR(optimizer_ft_18, step_size=7, gamma=0.1)

################## Inception
inception = models.inception_v3(pretrained=False)
optimizer_ft_inception = optim.Adam(inception.parameters(), lr=learning_rate)                  
exp_lr_scheduler_inception = lr_scheduler.StepLR(optimizer_ft_inception, step_size=7, gamma=0.1)

################# Alexnet
alexnet = models.alexnet(pretrained=False)
optimizer_ft_alexnet = optim.Adam(alexnet.parameters(), lr=learning_rate)                  
exp_lr_scheduler_alexnet = lr_scheduler.StepLR(optimizer_ft_alexnet, step_size=7, gamma=0.1)

################# Vgg
vgg16 = models.vgg16(pretrained=False)
optimizer_ft_vgg16 = optim.Adam(vgg16.parameters(), lr=learning_rate)                   
exp_lr_scheduler_vgg16 = lr_scheduler.StepLR(optimizer_ft_vgg16, step_size=7, gamma=0.1)

################# Squeezenet
squeezenet = models.squeezenet1_0(pretrained=False)
optimizer_ft_squeezenet = optim.Adam(squeezenet.parameters(), lr=learning_rate)                  
exp_lr_scheduler_squeezenet = lr_scheduler.StepLR(optimizer_ft_squeezenet, step_size=7, gamma=0.1)

################# densenet
densenet = models.densenet161(pretrained=False)
optimizer_ft_densenet = optim.Adam(densenet.parameters(), lr=learning_rate)                  
exp_lr_scheduler_densenet = lr_scheduler.StepLR(optimizer_ft_densenet, step_size=7, gamma=0.1)

if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  resnet152 = nn.DataParallel(resnet152)
  resnet18 = nn.DataParallel(resnet18)
  inception = nn.DataParallel(inception)
  alexnet = nn.DataParallel(alexnet)
  vgg16 = nn.DataParallel(vgg16)
  squeezenet = nn.DataParallel(squeezenet)
  densenet = nn.DataParallel(densenet)

if use_gpu:
   resnet152.cuda()
   resnet18.cuda()
   inception.cuda()
   alexnet.cuda()
   vgg16.cuda()
   squeezenet.cuda()
   densenet.cuda()



KeyboardInterrupt: ignored

In [ ]:
####################### Train resnet152 ###########################

resnet152 = train_model(resnet152, criterion, optimizer_ft, exp_lr_scheduler,
                           num_epochs=5)


In [ ]:
# re-initialise resnet 18
resnet18 = models.resnet18(pretrained=False)
optimizer_ft_18 = optim.Adam(resnet18.parameters(), lr=learning_rate)                   
exp_lr_scheduler_18 = lr_scheduler.StepLR(optimizer_ft_18, step_size=7, gamma=0.1)

if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  resnet18 = nn.DataParallel(resnet18)

if use_gpu:
   resnet18.cuda()

In [ ]:
####################### Train resnet18 ###########################

resnet18 = train_model(resnet18, criterion, optimizer_ft_18, exp_lr_scheduler_18,
                           num_epochs=20)

In [ ]:
####################### Train inception ###########################

inception = train_model(inception, criterion, optimizer_ft_inception, exp_lr_scheduler_inception,
                           num_epochs=1)

#inception not working with training function
########## ERROR: padding and the input size....

In [ ]:
alexnet = train_model(alexnet, criterion, optimizer_ft_alexnet, exp_lr_scheduler_alexnet,
                           num_epochs=10)

In [ ]:
####################### Train vgg16 ###########################

vgg16 = train_model(vgg16, criterion, optimizer_ft_vgg16, exp_lr_scheduler_vgg16,
                           num_epochs=10)

In [ ]:
####################### Train squeezenet ###########################

squeezenet = train_model(squeezenet, criterion, optimizer_ft_squeezenet, exp_lr_scheduler_squeezenet,
                           num_epochs=10)

In [ ]:
####################### Train densenet ###########################

densenet = train_model(densenet, criterion, optimizer_ft_densenet, exp_lr_scheduler_densenet,
                           num_epochs=10)

In [ ]:
############### Initialise ResNet50 

# try to see how resnet trains over more epochs, 18 seems to limit it

resnet50 = models.resnet50(pretrained=False)
optimizer_ft_50 = optim.Adam(resnet50.parameters(), lr=learning_rate)                   
exp_lr_scheduler_50 = lr_scheduler.StepLR(optimizer_ft_50, step_size=7, gamma=0.1)

if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  resnet50 = nn.DataParallel(resnet18)

if use_gpu:
   resnet50.cuda()

In [ ]:
############### Train ResNet50 

if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  resnet18 = nn.DataParallel(resnet18)

if use_gpu:
   resnet18.cuda()
  
resnet50 = train_model(resnet50, criterion, optimizer_ft_50, exp_lr_scheduler_50,
                           num_epochs=20)


# Model Inspection
Having trained a number of models and developed a relative understanding of their performance, now the better models are inspected in more detail to see how they are operating.

It was found that the models from torchvision were in a standard form equipped for the Imagenet datasets. This meant that:
* The output layer was a fully connected layer with >>6 outputs
* to make the prediction more accurate and model smaller, the final layer needed to be replaced with a linear layer with 6 outputs
* to correctly do this, the model is inspected by printing it, and then the final layer is edited by name and replaced

In [ ]:
# print the model to observe the final layer
print(resnet18)

In [ ]:
# how to change the prediction layer
resnet18.fc = nn.Linear(512, 6)

# print the model to observe the change
print(resnet18)

In [19]:
###### train the edited model with 6 outputs

resnet18 = models.resnet18(pretrained=False)
resnet18.fc = nn.Linear(512, 6)
optimizer_ft_18 = optim.Adam(resnet18.parameters(), lr=learning_rate)                  
exp_lr_scheduler_18 = lr_scheduler.StepLR(optimizer_ft_18, step_size=7, gamma=0.1)

if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  resnet18 = nn.DataParallel(resnet18)


if use_gpu:
   resnet18.cuda()
resnet18 = train_model(resnet18, criterion, optimizer_ft_18, exp_lr_scheduler_18,
                           num_epochs=30)

Epoch 0/29
----------
train Loss: 0.0419 Acc: 0.5033 in 0m 59s
val Loss: 0.0695 Acc: 0.4406 in 13m 6s

Epoch 1/29
----------
train Loss: 0.0353 Acc: 0.5905 in 0m 59s
val Loss: 0.0468 Acc: 0.4550 in 0m 22s

Epoch 2/29
----------
train Loss: 0.0332 Acc: 0.5990 in 0m 59s
val Loss: 0.0385 Acc: 0.5375 in 0m 23s

Epoch 3/29
----------
train Loss: 0.0310 Acc: 0.6358 in 0m 59s
val Loss: 0.0364 Acc: 0.6279 in 0m 22s

Epoch 4/29
----------
train Loss: 0.0311 Acc: 0.6312 in 0m 59s
val Loss: 0.0324 Acc: 0.6440 in 0m 23s

Epoch 5/29
----------
train Loss: 0.0295 Acc: 0.6548 in 0m 59s
val Loss: 0.0370 Acc: 0.5862 in 0m 23s

Epoch 6/29
----------
train Loss: 0.0263 Acc: 0.6982 in 0m 59s
val Loss: 0.0229 Acc: 0.7388 in 0m 23s

Epoch 7/29
----------
train Loss: 0.0248 Acc: 0.7067 in 0m 60s
val Loss: 0.0232 Acc: 0.7275 in 0m 23s

Epoch 8/29
----------
train Loss: 0.0242 Acc: 0.7090 in 0m 59s
val Loss: 0.0221 Acc: 0.7398 in 0m 23s

Epoch 9/29
----------
train Loss: 0.0237 Acc: 0.7211 in 0m 59s
val Loss: 

In [26]:
# try different loss criterion
criterion = nn.CrossEntropyLoss()

################## Inception
inception = models.inception_v3(pretrained=False)
optimizer_ft_inception = optim.Adam(inception.parameters(), lr=learning_rate)                   
exp_lr_scheduler_inception = lr_scheduler.StepLR(optimizer_ft_inception, step_size=7, gamma=0.1)


if torch.cuda.device_count() > 1 and multiGPU:
  print("Using", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  inception = nn.DataParallel(inception)


if use_gpu:
   inception.cuda()


In [ ]:
####################### Train inception ###########################

inception = train_model(inception, criterion, optimizer_ft_inception, exp_lr_scheduler_inception,
                           num_epochs=1)

#inception isn't liking this training function atm......
########## ERROR: something to do with padding and the input size....